In [3]:
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

In [4]:
data=pd.read_csv("../data/processed/clean_brand_data.csv")
data

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,report_id,caers_created_date,date_of_event,product_type,product,product_code,category,patient_age,age_units,sex,medra_preferred_terms,outcomes,brand
0,0,172934,1/1/2014,12/5/2013,SUSPECT,VALUED NATIONAL PINE NUTS,23,Nuts/Edible Seed,NaN,NaN,NaN,"DYSGEUSIA, HYPERSENSITIVITY",['Other Outcome'],VALUED NATIONAL
1,1,172937,1/1/2014,NaN,SUSPECT,"DAVID'S SUNFLOWER SEEDS, RANCH FLAVORED",23,Nuts/Edible Seed,NaN,NaN,NaN,"PAIN, MUCOSAL ULCERATION, TENDERNESS, BURNING ...","['Hospitalization', 'Patient Visited Healthcar...",DAVIDS SUNFLOWER
2,2,172939,1/1/2014,NaN,SUSPECT,"KASHI WHOLE WHEAT BISCUITS, ISLAND VANILLA",5,Cereal Prep/Breakfast Food,NaN,NaN,NaN,"VOMITING, ABDOMINAL PAIN, FEELING OF BODY TEMP...",['Other Outcome'],KASHI
3,3,172940,1/1/2014,NaN,SUSPECT,DANNON DANNON LITE & FIT GREEK YOGURT CHERRY,9,Milk/Butter/Dried Milk Prod,NaN,NaN,NaN,NAUSEA,['Other Outcome'],DANNON
4,4,172941,1/1/2014,12/16/2013,SUSPECT,COPPER RIVER KIPPERED ALASKA SILVER SALMON,16,Fishery/Seafood Prod,NaN,NaN,NaN,LACERATION,['Other Outcome'],COPPER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131253,131256,172888,12/31/2013,NaN,CONCOMITANT,FISH OIL (FISH OIL),54,Vit/Min/Prot/Unconv Diet(Human/Animal),87.0,year(s),F,"DYSPHAGIA, CHOKING",['Medically Important'],FISH OIL
131254,131257,172888,12/31/2013,NaN,SUSPECT,"CENTRUM SILVER WOMEN'S 50+ (MULTIMINERALS, MUL...",54,Vit/Min/Prot/Unconv Diet(Human/Animal),87.0,year(s),F,"DYSPHAGIA, CHOKING",['Medically Important'],CENTRUM SILVER
131255,131258,172889,12/31/2013,NaN,SUSPECT,"CENTRUM SILVER WOMEN'S 50+ (MULTIMINERALS, MUL...",54,Vit/Min/Prot/Unconv Diet(Human/Animal),86.0,year(s),F,CHOKING,['Medically Important'],CENTRUM SILVER
131256,131259,172891,12/31/2013,NaN,SUSPECT,"CENTRUM FLAVOR BURST(MUL TIMINERALS, MULTIVITA...",54,Vit/Min/Prot/Unconv Diet(Human/Animal),75.0,year(s),F,"ROTATOR CUFF SYNDROME, POOR QUALITY DRUG ADMIN...",['Medically Important'],CENTRUM FLAVOR


In [20]:
def symptom_counter(file_name: str, variable: int = 0):
    """This function will return a dictionary containing counts of each symptom present in data file under a given condition, 
    dictated by variable

    Args:
        file_name (str): The CSV file on which function will execute
        cosmetic (int): 0 -> all categories, all products
                        1 -> only for cosmetics as a categorie
                        2 -> only for quorn as a product

    Returns:
        (dictionary): A dictionary with keys as symptoms and values as total count
    """
    assert isinstance(file_name, str) and len(file_name)>0 , "file_name is either empty or not a string"
    assert isinstance(variable, int) and 0<=variable<=2, "variable is not an integer in the range [0,2]"
    dic=defaultdict(int)
    data=pd.read_csv(file_name)
    if variable==1: data=data.drop(data.index[(data['category']!='Cosmetics')])
    elif variable==2: data=data.drop(data.index[(data['brand']!='QUORN')])
    for dat in data["medra_preferred_terms"]:
        if dat== '' or pd.isnull(dat): continue
        lis=dat.split(",")
        for i in lis:
            i=i.strip()
            dic[i]+=1
    if variable==1:
        del dic["DEATH"] #Since this is probably an error made by doctors, it should be an outcome not a symptom
        del dic["INJURY"] #Since this is probably an error made by doctors, it should be an outcome not a symptom
    return dic

file_name="../data/processed/clean_brand_data.csv"
dic=symptom_counter(file_name,2)
print(dic)

defaultdict(<class 'int'>, {'VOMITING': 373, 'NAUSEA': 218, 'DIARRHOEA': 255, 'CHILLS': 28, 'ABDOMINAL PAIN': 47, 'CHEST PAIN': 6, 'HYPERSENSITIVITY': 26, 'FLUSHING': 1, 'ACTIVITIES OF DAILY LIVING IMPAIRED': 2, 'ABDOMINAL PAIN UPPER': 135, 'ANAPHYLACTIC SHOCK': 2, 'SWELLING': 11, 'SWELLING FACE': 8, 'LIP SWELLING': 12, 'PRURITUS': 25, 'HOT FLUSH': 3, 'GLOSSITIS': 1, 'RESPIRATORY TRACT CONGESTION': 1, 'DRY SKIN': 1, 'SALIVA ALTERED': 1, 'PHARYNGEAL OEDEMA': 6, 'RASH GENERALISED': 4, 'EYE SWELLING': 11, 'FEELING ABNORMAL': 5, 'MALAISE': 57, 'HEADACHE': 51, 'ABDOMINAL DISCOMFORT': 18, 'LACRIMATION INCREASED': 2, 'SNEEZING': 4, 'DEHYDRATION': 22, 'TACHYCARDIA': 3, 'ERYTHEMA': 11, 'COUGH': 7, 'FACE OEDEMA': 1, 'PARAESTHESIA ORAL': 1, 'RETCHING': 11, 'COLD SWEAT': 9, 'DIZZINESS': 46, 'FLATULENCE': 25, 'PAIN': 22, 'JOINT RANGE OF MOTION DECREASED': 1, 'HYPERHIDROSIS': 27, 'RASH': 17, 'RASH MACULAR': 3, 'DYSPNOEA': 43, 'PALPITATIONS': 4, 'PYREXIA': 10, 'OCULAR HYPERAEMIA': 2, 'WHEEZING': 8, '

In [21]:
def top_symptoms(dic,title):
    """Find and plot top symptoms in the dictionary based on count

    Args:
        dic (dict): Dictionary containing text-count pair

    Returns:
        [dictionary]: Top 5 symptoms with their count
    """
    assert isinstance(dic,dict) and len(dic)>0, "dic is not a nonempty dictionary"
    labels = []
    sizes = []
    counts=0
    top5=sorted(dic, key=dic.get, reverse=True)[:5]
    others=sorted(dic, key=dic.get, reverse=True)[5:]
    for i in others:counts+=dic[i]
    for i in top5:
        labels.append(i)
        sizes.append(dic[i])
    labels.append("OTHER")
    sizes.append(counts)
    fig = go.Figure(data=[go.Pie(labels=labels, values=sizes, hole=.3)])
    fig.update_layout(
        title=title,
        template=None,
        title_x=0.5,
        width=1000, 
        height=900,
        margin=dict(l=20, r=20, t=50, b=20),
        legend = dict(font = dict(size = 25, color = "black"))
        )
    fig.show()
    return top5

top5=top_symptoms(dic,"Symptoms for Cosmetics")

In [139]:
data=pd.read_csv("../data/processed/clean_brand_data.csv")
data["medra_preferred_terms"]=data["medra_preferred_terms"].str.split(",")
F_symp_vs_description=defaultdict(set)
M_symp_vs_description=defaultdict(set)
for i in range(data.shape[0]):
    #print(data["sex"])
    if type(data["sex"][i])!=float and data["sex"][i].strip()=="F":
        #print(type(data["medra_preferred_terms"][i]))
        if type(data["medra_preferred_terms"][i])==float:
            continue
        
        for j in range(len(data["medra_preferred_terms"][i])):
            if data["medra_preferred_terms"][i][j] == '' or pd.isnull(data["medra_preferred_terms"][i][j]) or type(data["category"][i])==float:
                continue
            descrip=data["category"][i].strip()
            symp=data["medra_preferred_terms"][i][j].strip()
            #if descrip not in symp_vs_product[symp]:
            F_symp_vs_description[symp].add(descrip)
    elif type(data["sex"][i])!=float and data["sex"][i].strip()=="M":
        #print(type(data["medra_preferred_terms"][i]))
        if type(data["medra_preferred_terms"][i])==float:
            continue
        
        for j in range(len(data["medra_preferred_terms"][i])):
            if data["medra_preferred_terms"][i][j] == '' or pd.isnull(data["medra_preferred_terms"][i][j]) or type(data["category"][i])==float:
                continue
            descrip=data["category"][i].strip()
            symp=data["medra_preferred_terms"][i][j].strip()
            #if descrip not in symp_vs_product[symp]:
            M_symp_vs_description[symp].add(descrip)


/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning:

Columns (1,6) have mixed types.Specify dtype option on import or set low_memory=False.



In [11]:
data=pd.read_csv("../data/processed/clean_brand_data.csv")
data["medra_preferred_terms"]=data["medra_preferred_terms"].str.split(",")
symp_vs_descrip=defaultdict(set)
descrip_count=defaultdict(int)
for i in range(data.shape[0]):
    #print(type(data["medra_preferred_terms"][i]))
    if type(data["medra_preferred_terms"][i])==float:
        continue
    #print(data["medra_preferred_terms"][i])
    for j in range(len(data["medra_preferred_terms"][i])):
        if data["medra_preferred_terms"][i][j] == '' or pd.isnull(data["medra_preferred_terms"][i][j]) or type(data["category"][i])==float:
            continue
        descrip=data["category"][i].strip()
        symp=data["medra_preferred_terms"][i][j].strip()
        #if descrip not in symp_vs_product[symp]:
        symp_vs_descrip[symp].add(descrip)
        if symp in top5:
            descrip_count[symp+"+"+descrip]+=1
#print(symp_vs_product)


/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning:

Columns (1,6) have mixed types.Specify dtype option on import or set low_memory=False.



In [12]:
sorted_decriptions=sorted(descrip_count, key=descrip_count.get, reverse=True)
max_count=5
top_symp_top_descrip=defaultdict(list)
top_symp_top_descrip_count=defaultdict(list)
for i in sorted_decriptions:
    [pre,post]=i.split("+")
    if len(top_symp_top_descrip[pre])<max_count:
        top_symp_top_descrip[pre].append(post)
        top_symp_top_descrip_count[pre].append(descrip_count[i])
print(top_symp_top_descrip)
print(top_symp_top_descrip_count)

defaultdict(<class 'list'>, {'OVARIAN CANCER': ['Cosmetics', 'Vit/Min/Prot/Unconv Diet(Human/Animal)'], 'NAUSEA': ['Vit/Min/Prot/Unconv Diet(Human/Animal)', 'Vegetables/Vegetable Products', 'Fishery/Seafood Prod', 'Nuts/Edible Seed', 'Soft Drink/Water'], 'VOMITING': ['Vit/Min/Prot/Unconv Diet(Human/Animal)', 'Vegetables/Vegetable Products', 'Nuts/Edible Seed', 'Fruit/Fruit Prod', 'Bakery Prod/Dough/Mix/Icing'], 'DIARRHOEA': ['Vit/Min/Prot/Unconv Diet(Human/Animal)', 'Vegetables/Vegetable Products', 'Nuts/Edible Seed', 'Fishery/Seafood Prod', 'Fruit/Fruit Prod'], 'ABDOMINAL PAIN': ['Vit/Min/Prot/Unconv Diet(Human/Animal)', 'Nuts/Edible Seed', 'Fishery/Seafood Prod', 'Vegetables/Vegetable Products', 'Bakery Prod/Dough/Mix/Icing']})
defaultdict(<class 'list'>, {'OVARIAN CANCER': [11746, 80], 'NAUSEA': [5982, 913, 823, 781, 716], 'VOMITING': [5312, 1343, 1047, 879, 848], 'DIARRHOEA': [4975, 1755, 1563, 1372, 1016], 'ABDOMINAL PAIN': [2883, 1109, 825, 813, 654]})


In [13]:

for i in top_symp_top_descrip:
    des=top_symp_top_descrip[i]
    print(des)
    vals=top_symp_top_descrip_count[i]
    print(vals)
    zipped_list=zip(des,vals)
    df = pd.DataFrame(zipped_list, columns = ['Description', 'No. of instances'])
    #fig = px.histogram(df, x='Description', y='No. of instances',title=str(i))
    #fig.show()

['Cosmetics', 'Vit/Min/Prot/Unconv Diet(Human/Animal)']
[11746, 80]
['Vit/Min/Prot/Unconv Diet(Human/Animal)', 'Vegetables/Vegetable Products', 'Fishery/Seafood Prod', 'Nuts/Edible Seed', 'Soft Drink/Water']
[5982, 913, 823, 781, 716]
['Vit/Min/Prot/Unconv Diet(Human/Animal)', 'Vegetables/Vegetable Products', 'Nuts/Edible Seed', 'Fruit/Fruit Prod', 'Bakery Prod/Dough/Mix/Icing']
[5312, 1343, 1047, 879, 848]
['Vit/Min/Prot/Unconv Diet(Human/Animal)', 'Vegetables/Vegetable Products', 'Nuts/Edible Seed', 'Fishery/Seafood Prod', 'Fruit/Fruit Prod']
[4975, 1755, 1563, 1372, 1016]
['Vit/Min/Prot/Unconv Diet(Human/Animal)', 'Nuts/Edible Seed', 'Fishery/Seafood Prod', 'Vegetables/Vegetable Products', 'Bakery Prod/Dough/Mix/Icing']
[2883, 1109, 825, 813, 654]


In [58]:
import plotly.express as px
for i in top_symp_top_descrip:
    des=top_symp_top_descrip[i]
    vals=top_symp_top_descrip_count[i]
    zipped_list=zip(des,vals)
    df = pd.DataFrame(zipped_list, columns = ['Category', 'No. of instances'])
    fig = px.histogram(df, x='Category', y='No. of instances',title=str(i))
    fig.show()


In [15]:
from plotly.subplots import make_subplots
traces=[]
counter=0
colors=['red','violet','blue','yellow','purple']
for i in top_symp_top_descrip:
    des=top_symp_top_descrip[i]
    vals=top_symp_top_descrip_count[i]
    #zipped_list=zip(des,vals)
    #df = pd.DataFrame(zipped_list, columns = ['Description', 'No. of instances'])
    #fig = px.histogram(df, x='Description', y='No. of instances',title=str(i))

    df = pd.DataFrame(dict(r=vals,theta=des))
    fig = px.line_polar(df, r='r', theta='theta', line_close=True,title=i)
    fig.update_traces(fill='toself')
    fig.update_layout( title_x=0.5)
    counter+=1
    #traces.append(fig)
    
    fig.show()
    #fig.write_image("images/fig"+str(counter)+".png")

In [16]:
top_symptoms_vs_descrip={}
for symptom in top5:
    top_symptoms_vs_descrip[symptom]=symp_vs_descrip[symptom]

print((top_symptoms_vs_descrip.values()))
##calculate top 5 symptoms and their top 5 descriptions

dict_values([{'Ready to Serve Formula', 'Snack Food Item', 'G Medical Foods, N.E.C.', 'Alcoholic Beverage', 'Ice Cream Prod', 'Soft Drink/Water', 'Gelatin/Rennet/Pudding Mix/Pie Filling', 'Cheese/Cheese Prod', 'Medical Foods, N.E.C.', 'Vegetables/Vegetable Products', 'Baby Food Products', 'Soup', 'Macaroni/Noodle Prod', 'Beverage Bases/Conc/Nectar', 'Vegetable Oils', 'Whole Grain/Milled Grain Prod/Starch', 'Milk/Butter/Dried Milk Prod', 'Filled Milk/Imit Milk Prod', 'Prep Salad Prod', 'Meat, Meat Products and Poultry', 'Choc/Cocoa Prod', 'Vegetable Protein Prod', 'Miscellaneous Food Related Items', 'Color Additiv Food/Drug/Cosmetic', 'Cereal Prep/Breakfast Food', 'Coffee/Tea', 'Candy W/O Choc/Special/Chew Gum', 'Powder Formula', 'Nuts/Edible Seed', 'Food Additives (Human Use)', 'Egg/Egg Prod', 'Spices, Flavors And Salts', 'Mult Food Dinner/Grav/Sauce/Special', 'Dressing/Condiment', 'Fruit/Fruit Prod', 'Bakery Prod/Dough/Mix/Icing', 'Dietary Conventional Foods/Meal Replacements', 'Food 

In [17]:
M_only_symp_vc_descr=defaultdict(set)
for key, values in M_symp_vs_description.items():
    for value in values:
        if key in F_symp_vs_description and value in F_symp_vs_description[key]:
            continue
       
        M_only_symp_vc_descr[key].add(value)

F_only_symp_vc_descr=defaultdict(set)
for key, values in F_symp_vs_description.items():
    for value in values:
        if key in M_symp_vs_description and value in M_symp_vs_description[key]:
            continue
       
        F_only_symp_vc_descr[key].add(value)

Top symptoms include ovarian cancer even though only females can get it. Mean there are products out there causing high probability of ovarian cancer

Formula causing vomiting for males but not females AND G medical cauing for females but not males

Only for males is alcohol causing a problem -> females dont drink as much or female anatomy not affected (unlikely reasn)


In [62]:
total_list=[]

for i in top5:
    val=[]
    if len(M_only_symp_vc_descr[i])>0:
        if len(F_only_symp_vc_descr[i])>0:
            val=[i,list(M_only_symp_vc_descr[i]), list(F_only_symp_vc_descr[i])]
        else:
            val=[i,list(M_only_symp_vc_descr[i]), ""]
    else:
        if len(F_only_symp_vc_descr[i])>0:
            val=[i,"", list(F_only_symp_vc_descr[i])]
        else:
            val=[i,"", ""]

    total_list.append(val)

df = pd.DataFrame(total_list, columns = ["Symptoms", "Unique_Description_for_Male", "Unique_Description_for_Female"])
rowEvenColor = 'lavender'
rowOddColor = 'pink'
fig = go.Figure(data=[go.Table(
    columnwidth = [80,200,1000],
    header=dict(values=list(df.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df.Symptoms, df.Unique_Description_for_Male, df.Unique_Description_for_Female],
               fill_color=[[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
               align='left'))
])
fig.show()

In [35]:
data=pd.read_csv("../data/processed/clean_brand_data.csv")
data["category"]=data["category"].str.strip()
grouped_desc=data.groupby("category")#.str.strip()
symptom_list=['DIARRHOEA', 'VOMITING', 'NAUSEA', 'ABDOMINAL PAIN']

grouped_desc_vit=grouped_desc.get_group("Cosmetics")
grouped_desc_vit["medra_preferred_terms"]=grouped_desc_vit["medra_preferred_terms"].str.split(",")

grouped_desc_vit= grouped_desc_vit.explode("medra_preferred_terms").drop_duplicates()
grouped_desc_vit["medra_preferred_terms"]=grouped_desc_vit["medra_preferred_terms"].str.strip()

grouped_desc_vit2=grouped_desc_vit.groupby("medra_preferred_terms")
grouped_desc_vit=grouped_desc_vit2.get_group('OVARIAN CANCER')
list_of_counts=grouped_desc_vit["brand"].value_counts()
list_of_counts=list_of_counts.reset_index()
i = list_of_counts[list_of_counts["index"]== 'EXEMPTION'].index
list_of_counts=list_of_counts.drop(i)
#print(list_of_counts)
prods=list(list_of_counts["index"])[0:5]
quants=list(list_of_counts["brand"])[0:5]

df = pd.DataFrame(dict(r=quants,theta=prods))
fig = px.line_polar(df, r='r', theta='theta', line_close=True,title='OVARIAN CANCER')
fig.update_traces(fill='toself')
fig.update_layout( title_x=0.5)

fig.show()

for symp in symptom_list:
    grouped_desc_vit=grouped_desc.get_group('Vit/Min/Prot/Unconv Diet(Human/Animal)')
    grouped_desc_vit["medra_preferred_terms"]=grouped_desc_vit["medra_preferred_terms"].str.split(",")

    grouped_desc_vit= grouped_desc_vit.explode("medra_preferred_terms").drop_duplicates()
    grouped_desc_vit["medra_preferred_terms"]=grouped_desc_vit["medra_preferred_terms"].str.strip()

    grouped_desc_vit2=grouped_desc_vit.groupby("medra_preferred_terms")
    grouped_desc_vit=grouped_desc_vit2.get_group(symp)
    list_of_counts=grouped_desc_vit["brand"].value_counts()
    list_of_counts=list_of_counts.reset_index()
    i = list_of_counts[list_of_counts["index"]== 'EXEMPTION 4'].index
    list_of_counts=list_of_counts.drop(i)
    prods=list(list_of_counts["index"])[:5]
    quants=list(list_of_counts["brand"])[:5]
    df = pd.DataFrame(dict(r=quants,theta=prods))
    fig = px.line_polar(df, r='r', theta='theta', line_close=True,title=symp)
    fig.update_traces(fill='toself')
    fig.update_layout( title_x=0.5)

    fig.show()

#print()
#print(sum(list_of_counts["product"]))


/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning:

Columns (1,6) have mixed types.Specify dtype option on import or set low_memory=False.

<ipython-input-35-c5981cb9664e>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



<ipython-input-35-c5981cb9664e>:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [39]:
import plotly.graph_objects as go

fig = go.Figure()
symptom_list=['DIARRHOEA', 'VOMITING', 'NAUSEA', 'ABDOMINAL PAIN']
data=pd.read_csv("../data/processed/clean_brand_data.csv")
data["category"]=data["category"].str.strip()
grouped_desc=data.groupby("category")#.str.strip()

for symp in symptom_list:
    grouped_desc_vit=grouped_desc.get_group('Vit/Min/Prot/Unconv Diet(Human/Animal)')
    grouped_desc_vit["medra_preferred_terms"]=grouped_desc_vit["medra_preferred_terms"].str.split(",")

    grouped_desc_vit= grouped_desc_vit.explode("medra_preferred_terms").drop_duplicates()
    grouped_desc_vit["medra_preferred_terms"]=grouped_desc_vit["medra_preferred_terms"].str.strip()

    grouped_desc_vit2=grouped_desc_vit.groupby("medra_preferred_terms")
    grouped_desc_vit=grouped_desc_vit2.get_group(symp)
    list_of_counts=grouped_desc_vit["brand"].value_counts()
    list_of_counts=list_of_counts.reset_index()
    i = list_of_counts[list_of_counts["index"]== 'EXEMPTION 4'].index
    list_of_counts=list_of_counts.drop(i)
    prods=list(list_of_counts["index"])[:5]
    quants=list(list_of_counts["brand"])[:5]
    zipped=zip(prods,quants)
    prods2=[]
    quants2=[]
    for p,q in zipped:
      prods2.append(p)
      quants2.append(q)

    fig.add_trace(go.Scatterpolar(
      r=quants2,
      theta=prods2,
      fill='toself',
      name=symp
    ))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 200]
    )),
  showlegend=True,
  title="Top Vitamins vs Top Symptoms",
  title_x=0.5

)

fig.show()

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning:

Columns (1,6) have mixed types.Specify dtype option on import or set low_memory=False.

<ipython-input-39-78fa4b3c9d21>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [23]:
def top_vitamins_symptom_distribution(file_name):
    """This function will plot a histogram for Reported Cases vs Products, where Products are the top vitamin products causing the
    top 5 symptoms

    Args:
        file_name (str): Name of the file used
    """
    assert isinstance(file_name,str) and len(file_name)>0, "file_name is either empty or a non string"
    fig = go.Figure()
    symptom_list=['DIARRHOEA', 'VOMITING', 'NAUSEA', 'ABDOMINAL PAIN']
    data=pd.read_csv(file_name)
    data["category"]=data["category"].str.strip()
    grouped_desc=data.groupby("category")
    add_on_list=[]
    for symp in symptom_list:
        grouped_desc_vit=grouped_desc.get_group('Vit/Min/Prot/Unconv Diet(Human/Animal)')
        grouped_desc_vit["medra_preferred_terms"]=grouped_desc_vit["medra_preferred_terms"].str.split(",")
        grouped_desc_vit= grouped_desc_vit.explode("medra_preferred_terms").drop_duplicates()
        grouped_desc_vit["medra_preferred_terms"]=grouped_desc_vit["medra_preferred_terms"].str.strip()
        grouped_desc_vit2=grouped_desc_vit.groupby("medra_preferred_terms")
        grouped_desc_vit=grouped_desc_vit2.get_group(symp)
        list_of_counts=grouped_desc_vit["brand"].value_counts()
        list_of_counts=list_of_counts.reset_index()
        i = list_of_counts[list_of_counts["index"]== 'EXEMPTION 4'].index
        list_of_counts=list_of_counts.drop(i)
        prods=list(list_of_counts["index"])[:5]
        quants=list(list_of_counts["brand"])[:5]
        lis_symp=[symp]*5
        zipped=zip(prods,quants,lis_symp)
        for lis in zipped:
            add_on_list.append(list(lis))
        
    df=pd.DataFrame(list(add_on_list),columns=['Products','Reported Cases','Symptom'])
    fig = px.histogram(df, x="Products", y="Reported Cases", color="Symptom",title="Top Vitamin Products Causing Symptoms ")
    fig.update_layout(
        barmode="stack",
        bargap=0.1)
    fig.show()

file_name="../data/processed/clean_brand_data.csv"
top_vitamins_symptom_distribution(file_name)

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning:

Columns (1,6) have mixed types.Specify dtype option on import or set low_memory=False.

<ipython-input-23-239930b28a37>:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

